# Data cleaning

Data cleaning of credit_card_balance.csv and installments_payments.csv.

In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import StandardScaler

## Credit card balance

- Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.

- This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

CUSTOMER RISK PROFILE

- Number of Loans per Customer
- Rate at which Loan is paid back by customer - No of instalments per customer per loan
- How much did the Customer load a Credit line?
- How many times did the Customer miss the minimum payment?
- What is the average number of days did Customer go past due date?
- What fraction of minimum payments were missed?

CUSTOMER BEHAVIOUR PATTERNS

- Cash withdrawals VS Overall Spending ratio
- Average number of drawings per customer - Total Drawings / Number of Drawings

In [2]:
# Load data

credit_card_balance = pd.read_csv('credit_card_balance.csv', sep=',')

credit_card_balance.head(10)

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.500,0.0,877.500,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.000,0.0,0.000,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.000,0.0,0.000,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.000,0.0,0.000,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.000,0.0,11547.000,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0
5,2646502,380010,-7,82903.815,270000,0.0,0.000,0.0,0.000,4449.105,...,82773.315,82773.315,0.0,0,0.0,0.0,2.0,Active,7,0
6,1079071,171320,-6,353451.645,585000,67500.0,67500.000,0.0,0.000,14684.175,...,351881.145,351881.145,1.0,1,0.0,0.0,6.0,Active,0,0
7,2095912,118650,-7,47962.125,45000,45000.0,45000.000,0.0,0.000,0.000,...,47962.125,47962.125,1.0,1,0.0,0.0,51.0,Active,0,0
8,2181852,367360,-4,291543.075,292500,90000.0,289339.425,0.0,199339.425,130.500,...,286831.575,286831.575,3.0,8,0.0,5.0,3.0,Active,0,0
9,1235299,203885,-5,201261.195,225000,76500.0,111026.700,0.0,34526.700,6338.340,...,197224.695,197224.695,3.0,9,0.0,6.0,38.0,Active,0,0


In [3]:
# Identify duplicate rows

duplicate_rows = credit_card_balance[credit_card_balance.duplicated()]
len(duplicate_rows)

0

### Feature selection

In [4]:
# Feature selection

credit_card_balance.loc[0:10, ["SK_ID_PREV", "SK_ID_CURR", "AMT_PAYMENT_CURRENT", "AMT_PAYMENT_TOTAL_CURRENT"]]

,SK_ID_PREV,SK_ID_CURR,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT
0,2562384,378907,1800.000,1800.000
1,2582071,363914,2250.000,2250.000
2,1740877,371185,2250.000,2250.000
3,1389973,337855,11925.000,11925.000
4,1891521,126868,27000.000,27000.000
5,2646502,380010,3825.000,3825.000
6,1079071,171320,15750.000,15750.000
7,2095912,118650,264.690,0.000
8,2181852,367360,4093.515,4093.515
9,1235299,203885,45000.000,45000.000


In [5]:
credit_card_balance.loc[credit_card_balance["AMT_PAYMENT_CURRENT"] < credit_card_balance["AMT_PAYMENT_TOTAL_CURRENT"],
                        :].loc[:, ["AMT_PAYMENT_CURRENT", "AMT_PAYMENT_TOTAL_CURRENT"]]

,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT


AMT_PAYMENT_CURRENT is never lower than AMT_PAYMENT_TOTAL_CURRENT (it is always higher or equal, or they are both null), which means that AMT_PAYMENT_CURRENT contains the value in AMT_PAYMENT_TOTAL_CURRENT. Therefore, we can drop that column.

In [6]:
credit_card_balance.loc[credit_card_balance["AMT_DRAWINGS_CURRENT"] < credit_card_balance["AMT_DRAWINGS_ATM_CURRENT"], :].loc[:, ["AMT_DRAWINGS_CURRENT",
                                                                                                                                  "AMT_DRAWINGS_ATM_CURRENT"]]

,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_ATM_CURRENT
438776,-1687.50,0.0
747302,-519.57,0.0
3284667,-6211.62,0.0


Same thing for AMT_DRAWINGS_CURRENT and AMT_DRAWINGS_ATM_CURRENT. Even though the amount of drawings in the ATM may be zero, the amount of drawings in total can be negative - when the amount of money drawing during a month by a client is lower than zero, it means that the client has made a payment to the bank that is greater than the outstanding balance on their credit card.

Therefore, we can drop AMT_DRAWINGS_ATM_CURRENT, because AMT_DRAWINGS_CURRENT contains all the information. Same thing applies to AMT_DRAWINGS_POS_CURRENT. 

In [7]:
credit_card_balance.loc[0:10, ["AMT_DRAWINGS_CURRENT", "AMT_DRAWINGS_OTHER_CURRENT"]]

,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT
0,877.500,0.0
1,2250.000,0.0
2,0.000,0.0
3,2250.000,0.0
4,11547.000,0.0
5,0.000,0.0
6,67500.000,0.0
7,45000.000,0.0
8,289339.425,0.0
9,111026.700,0.0


In [8]:
credit_card_balance.loc[credit_card_balance["AMT_DRAWINGS_CURRENT"] < credit_card_balance["AMT_DRAWINGS_OTHER_CURRENT"], :].loc[:, ["AMT_DRAWINGS_CURRENT", "AMT_DRAWINGS_OTHER_CURRENT"]]

,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT
438776,-1687.50,0.0
747302,-519.57,0.0
3284667,-6211.62,0.0


We can also drop this column.

Regarding AMT_RECEIVABLE_PRINCIPAL, AMT_RECIVABLE and AMT_TOTAL_RECEIVABLE, since AMT_TOTAL_RECEIVABLE is the total amount receivable on the previous credit, that contains the number we need and we can drop the other two columns.

Considering the columns we decided to drop, regarding the 4 columns with the number of drawings in the month in matter, it only makes sense to keep CNT_DRAWINGS_CURRENT, because that's what we can connect to the information that wasn't dropped. Also, that column contains total number of drawings, while the other 3 contained very specific drawings.

In [9]:
# Drop the columns

credit_card_balance.drop(['AMT_PAYMENT_TOTAL_CURRENT', 'AMT_DRAWINGS_ATM_CURRENT',
                          'AMT_DRAWINGS_POS_CURRENT', 'AMT_DRAWINGS_OTHER_CURRENT',
                          "AMT_RECEIVABLE_PRINCIPAL", "AMT_RECIVABLE",
                          "CNT_DRAWINGS_ATM_CURRENT", "CNT_DRAWINGS_OTHER_CURRENT",
                          "CNT_DRAWINGS_POS_CURRENT", "AMT_INST_MIN_REGULARITY", 
                         "SK_DPD", "SK_DPD_DEF"],
                         axis=1, inplace=True)

In [10]:
credit_card_balance

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS
0,2562384,378907,-6,56.970,135000,877.5,1800.00,0.000,1,35.0,Active
1,2582071,363914,-1,63975.555,45000,2250.0,2250.00,64875.555,1,69.0,Active
2,1740877,371185,-7,31815.225,450000,0.0,2250.00,31460.085,0,30.0,Active
3,1389973,337855,-4,236572.110,225000,2250.0,11925.00,233048.970,1,10.0,Active
4,1891521,126868,-1,453919.455,450000,11547.0,27000.00,453919.455,1,101.0,Active
...,...,...,...,...,...,...,...,...,...,...,...
3840307,1036507,328243,-9,0.000,45000,0.0,NaN,0.000,0,0.0,Active
3840308,1714892,347207,-9,0.000,45000,0.0,1879.11,0.000,0,23.0,Active
3840309,1302323,215757,-9,275784.975,585000,270000.0,375750.00,273093.975,2,18.0,Active
3840310,1624872,430337,-10,0.000,450000,0.0,NaN,0.000,0,0.0,Active


### Dealing with missing values

In [11]:
# Deal with missing values

n_null_dict = {}

for column in credit_card_balance.columns:
    null_sum = credit_card_balance.isnull()[column].sum()
    n_null_dict[column] = null_sum

n_null = pd.DataFrame(n_null_dict, index = ["Number of missing values"])

In [12]:
n_null

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS
Number of missing values,0,0,0,0,0,0,767988,0,0,305236,0


### Final pipeline to transform data and get new dataframe:

In [13]:
num_variables = [col for col in credit_card_balance.columns if col != 'NAME_CONTRACT_STATUS' and col != 'SK_ID_PREV' and col != 'SK_ID_CURR']

cat_variables = ["NAME_CONTRACT_STATUS"]

In [14]:
num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ])
cat_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="most_frequent")),
        ('cat_encoder', OneHotEncoder())
        ])

In [15]:
preprocess_pipeline = ColumnTransformer([
        ("num_pipeline", num_pipeline, num_variables),
        ("cat_pipeline", cat_pipeline, cat_variables)
])

In [16]:
preprocess_pipeline.fit(credit_card_balance)

preprocessed_credit_card_balance = preprocess_pipeline.transform(credit_card_balance)

In [17]:
cat_feature_names = list(preprocess_pipeline.named_transformers_['cat_pipeline'].named_steps['cat_encoder'].get_feature_names(cat_variables))
feature_names = num_variables + cat_feature_names
feature_names

/Users/patriciafonseca/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['MONTHS_BALANCE',
 'AMT_BALANCE',
 'AMT_CREDIT_LIMIT_ACTUAL',
 'AMT_DRAWINGS_CURRENT',
 'AMT_PAYMENT_CURRENT',
 'AMT_TOTAL_RECEIVABLE',
 'CNT_DRAWINGS_CURRENT',
 'CNT_INSTALMENT_MATURE_CUM',
 'NAME_CONTRACT_STATUS_Active',
 'NAME_CONTRACT_STATUS_Approved',
 'NAME_CONTRACT_STATUS_Completed',
 'NAME_CONTRACT_STATUS_Demand',
 'NAME_CONTRACT_STATUS_Refused',
 'NAME_CONTRACT_STATUS_Sent proposal',
 'NAME_CONTRACT_STATUS_Signed']

In [18]:
transformed_credit_card_balance_df = pd.DataFrame(preprocessed_credit_card_balance, columns=feature_names)
transformed_credit_card_balance_df = pd.concat([credit_card_balance[["SK_ID_CURR", 'SK_ID_PREV']],transformed_credit_card_balance_df], axis=1)
transformed_credit_card_balance_df

,SK_ID_CURR,SK_ID_PREV,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
0,378907,2562384,-6.0,56.970,135000.0,877.5,1800.00,0.000,1.0,35.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,363914,2582071,-1.0,63975.555,45000.0,2250.0,2250.00,64875.555,1.0,69.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,371185,1740877,-7.0,31815.225,450000.0,0.0,2250.00,31460.085,0.0,30.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,337855,1389973,-4.0,236572.110,225000.0,2250.0,11925.00,233048.970,1.0,10.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,126868,1891521,-1.0,453919.455,450000.0,11547.0,27000.00,453919.455,1.0,101.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840307,328243,1036507,-9.0,0.000,45000.0,0.0,2702.70,0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840308,347207,1714892,-9.0,0.000,45000.0,0.0,1879.11,0.000,0.0,23.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840309,215757,1302323,-9.0,275784.975,585000.0,270000.0,375750.00,273093.975,2.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3840310,430337,1624872,-10.0,0.000,450000.0,0.0,2702.70,0.000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Transforming dataset into dataset with one row per client 

In [19]:
credit_card_balance_grouped = transformed_credit_card_balance_df.groupby(["SK_ID_CURR", "SK_ID_PREV"]).agg({
    'AMT_BALANCE': 'mean',
    'AMT_CREDIT_LIMIT_ACTUAL': 'mean',
    'AMT_DRAWINGS_CURRENT': 'mean',
    'AMT_PAYMENT_CURRENT': 'mean',
    'AMT_TOTAL_RECEIVABLE': 'mean',
    'CNT_DRAWINGS_CURRENT': 'mean',
    'CNT_INSTALMENT_MATURE_CUM': 'mean',
    'NAME_CONTRACT_STATUS_Active': 'max',
    'NAME_CONTRACT_STATUS_Approved': 'max',
    'NAME_CONTRACT_STATUS_Completed': 'max',
    'NAME_CONTRACT_STATUS_Demand': 'max',
    'NAME_CONTRACT_STATUS_Refused': 'max',
    'NAME_CONTRACT_STATUS_Sent proposal': 'max',
    'NAME_CONTRACT_STATUS_Signed': 'max'
})

credit_card_balance_grouped = credit_card_balance_grouped.groupby("SK_ID_CURR").agg({
    'AMT_BALANCE': 'mean',
    'AMT_CREDIT_LIMIT_ACTUAL': 'mean',
    'AMT_DRAWINGS_CURRENT': 'mean',
    'AMT_PAYMENT_CURRENT': 'mean',
    'AMT_TOTAL_RECEIVABLE': 'mean',
    'CNT_DRAWINGS_CURRENT': 'mean',
    'CNT_INSTALMENT_MATURE_CUM': 'mean',
    'NAME_CONTRACT_STATUS_Active': 'sum',
    'NAME_CONTRACT_STATUS_Approved': 'sum',
    'NAME_CONTRACT_STATUS_Completed': 'sum',
    'NAME_CONTRACT_STATUS_Demand': 'sum',
    'NAME_CONTRACT_STATUS_Refused': 'sum',
    'NAME_CONTRACT_STATUS_Sent proposal': 'sum',
    'NAME_CONTRACT_STATUS_Signed': 'sum'
})


### Final dataset:

In [20]:
credit_card_balance_grouped

,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_CURRENT,AMT_PAYMENT_CURRENT,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS_Active,NAME_CONTRACT_STATUS_Approved,NAME_CONTRACT_STATUS_Completed,NAME_CONTRACT_STATUS_Demand,NAME_CONTRACT_STATUS_Refused,NAME_CONTRACT_STATUS_Sent proposal,NAME_CONTRACT_STATUS_Signed
SK_ID_CURR,,,,,,,,,,,,,,
100006,0.000000,270000.000000,0.000000,2702.700000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
100011,54482.111149,164189.189189,2432.432432,4843.064189,54433.179122,0.054054,25.621622,1.0,0.0,0.0,0.0,0.0,0.0,0.0
100013,18159.919219,131718.750000,5953.125000,7168.346250,18101.079844,0.239583,18.447917,1.0,0.0,0.0,0.0,0.0,0.0,0.0
100021,0.000000,675000.000000,0.000000,2702.700000,0.000000,0.000000,0.000000,1.0,0.0,1.0,0.0,0.0,0.0,0.0
100023,0.000000,135000.000000,0.000000,2702.700000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456244,131834.730732,296341.463415,26842.388049,32720.544878,130767.060732,1.365854,13.634146,1.0,0.0,1.0,0.0,0.0,0.0,0.0
456246,13136.731875,135000.000000,15199.256250,16768.828125,12897.894375,2.500000,3.500000,1.0,0.0,0.0,0.0,0.0,0.0,0.0
456247,23216.396211,144000.000000,2149.506474,4883.755263,23128.243105,0.147368,26.494737,1.0,0.0,0.0,0.0,0.0,0.0,0.0
